Stage 1: Clean up the code and modulize
1. Simplified VAE util
2. Sample and dock mol gen Script
3. Docking bash script
4. Score Processing script
5. Fingerprint script
6. Clustering script
7. Wrapper bash script <br>
Stage 2: Performance improvement <br>
Maybe compile most of the functions and write and compile the wrapper in C++?

In [1]:
import sys
import os
sys.path.append('/home/ziqiaoxu/Sample_Dock_DnD/jtvae/')
sys.path.append('/home/ziqiaoxu/Sample_Dock_DnD/SnD/')
from sampler import hyperparam_loader

from mol_tree import Vocab
from jtnn_vae import JTNNVAE
import torch

In [2]:
import sys
from SnD import dock, sort_pose, save_pose, smiles_to_sdfile

In [30]:
p = hyperparam_loader('test_hyper.param')


########### Parameters Loaded as Below ###########

receptor_name : CDK2-5IEV
receptor_file : ./targets/CDK2_5IEV/5IEV.mol2
ligand_file : ./targets/CDK2_5IEV/Roniciclib.sd
cavity_protocol : rbcavity -was -d -r
docking_prm : dock.prm
npose : 100
prefix : pose_docked_
sort_by : SCORE.INTER
vocab_loc : ./models/moses-h450z56/vocab.txt
hidden_size : 450
latent_size : 56
depthT : 20
depthG : 3
model_loc : ./models/moses-h450z56/model.iter-400000
ncycle : 5
ndesign : 20
seed_smi : C1=CC=CC=C1

##################################################



In [31]:
vocab = Vocab(p.vocab)
jtvae = JTNNVAE(vocab, p.hidden_size, p.latent_size, p.depthT, p.depthG)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
jtvae.load_state_dict(torch.load(p.model_loc, map_location=device))

<All keys matched successfully>

In [32]:
prmfile = os.path.abspath('./test/CDK2/cavity/pocket_docking.prm')
wd = os.path.abspath('./test/CDK2/')

In [ ]:
## VAE encoding and decoding on the first seeding smiles
design_list = jtvae.smiles_gen(p.seed_smi, p.ndesign)

## Main loop: VAE on subsequent returned compounds
for j in range(p.ncycle):
    
    design_dir = os.path.abspath(os.path.join(wd,'cycle_%s'%j))
    docking_dir = os.path.abspath(os.path.join(design_dir, 'docking'))
    try: os.makedirs(docking_dir)
    except FileExistsError: print(docking_dir,'Overwritten')
    
    ## write .sdf file and get ligs file names
    ligs = smiles_to_sdfile(design_list,design_dir)
    
    ## Generate docking scores from .sd files
    dock(ligs, docking_dir, prmfile, p.docking_prm, p.npose, p.prefix)
    ranked_poses = sort_pose(docking_dir, p.sort_by, p.prefix)
    save_pose(ranked_poses, design_dir)
    
    ## announce the winner
    best_energy, name, best_mol = ranked_poses[0]
    best_smi = best_mol.GetProp('SMILES')
    print("[INFO] Cycle %s: %s %s kcal/mol          "%(j, best_smi, best_energy))
    
    ## generate for next cycle
    design_list = jtvae.smiles_gen(best_smi, p.ndesign)

In [35]:
smi = jtvae.smiles_gen('Brc1cnn(-c2ccc(-c3nc[nH]n3)cc2)c1',20)